In [20]:
!pip install sktime
import glob
import joblib
import numpy as np
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
import sktime 
from sktime.datatypes._panel._convert import from_3d_numpy_to_nested

In [131]:
#imports
from sktime.utils.data_io import load_from_tsfile_to_dataframe
!pip install tsfresh
from tsfresh.feature_extraction import extract_features
from tsfresh.feature_extraction import settings
from sktime.transformations.panel.tsfresh import TSFreshFeatureExtractor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
import numpy as np
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [98]:
# Getting data from TS file
features=['USFLUX', 'TOTUSJH', 'ABSNJZH', 'SAVNCPP', 'TOTBSQ', 'TOTPOT', 'TOTUSJZ']
X_features,Y_target=load_from_tsfile_to_dataframe(r"E:\USA\GSU\Fall 2021\FDS\Project\latest\MVTS_Data\MVTS_flares_data\MVTS_flares_data_updated.ts",
                                           return_separate_X_and_y=True,)

X_features.columns=features
headDF = X_features.head()
headDF


,USFLUX,TOTUSJH,ABSNJZH,SAVNCPP,TOTBSQ,TOTPOT,TOTUSJZ
0,0 6.499284e+22 1 6.482391e+22 2 6....,0 3948.598770 1 3948.485756 2 3993...,0 180.722345 1 196.561268 2 228.12...,0 8.038467e+12 1 9.172823e+12 2 1....,0 5.511007e+10 1 5.508232e+10 2 5....,0 7.826781e+23 1 7.888169e+23 2 7....,0 7.948180e+13 1 7.966682e+13 2 8....
1,0 2.048862e+22 1 2.035425e+22 2 2....,0 1832.189161 1 1822.038319 2 1823...,0 57.873728 1 71.677487 2 56.09...,0 9.661905e+12 1 9.225191e+12 2 8....,0 2.031790e+10 1 2.022157e+10 2 2....,0 3.633399e+23 1 3.628075e+23 2 3....,0 4.170312e+13 1 4.117412e+13 2 4....
2,0 1.635331e+22 1 1.628852e+22 2 1....,0 972.192133 1 956.465485 2 960.92...,0 133.000377 1 138.407592 2 132.14...,0 2.790539e+12 1 2.368424e+12 2 3....,0 1.456621e+10 1 1.453914e+10 2 1....,0 2.292432e+23 1 2.277908e+23 2 2....,0 2.053599e+13 1 2.004484e+13 2 2....
3,0 4.854274e+22 1 4.874232e+22 2 4....,0 3958.624542 1 4062.845727 2 4053...,0 672.665536 1 691.311500 2 721.05...,0 4.502288e+13 1 4.517309e+13 2 4....,0 6.045770e+10 1 6.059242e+10 2 6....,0 1.154610e+24 1 1.157003e+24 2 1....,0 7.653851e+13 1 7.829640e+13 2 7....
4,0 2.047216e+22 1 2.059660e+22 2 2....,0 1873.556961 1 1888.526741 2 1855...,0 59.804189 1 37.324194 2 29.11030...,0 7.269096e+12 1 7.452472e+12 2 8....,0 2.134141e+10 1 2.135078e+10 2 2....,0 3.686462e+23 1 3.677807e+23 2 3....,0 3.933330e+13 1 4.012606e+13 2 3....


In [101]:
#Adding features to time series data
parameterForMean = settings.MinimalFCParameters()

keys_to_remove = ["length", "sum_values","variance","root_mean_square","maximum","minimum"]
for key in keys_to_remove:
  del parameterForMean[key]
parameterForMean

transformer = TSFreshFeatureExtractor(default_fc_parameters=parameterForMean)
extracted_features = transformer.fit_transform(X_features)
extracted_features.head()

Feature Extraction: 100%|██████████| 5/5 [03:35<00:00, 43.18s/it]


,ABSNJZH__median,ABSNJZH__mean,ABSNJZH__standard_deviation,SAVNCPP__median,SAVNCPP__mean,SAVNCPP__standard_deviation,TOTBSQ__median,TOTBSQ__mean,TOTBSQ__standard_deviation,TOTPOT__median,...,TOTPOT__standard_deviation,TOTUSJH__median,TOTUSJH__mean,TOTUSJH__standard_deviation,TOTUSJZ__median,TOTUSJZ__mean,TOTUSJZ__standard_deviation,USFLUX__median,USFLUX__mean,USFLUX__standard_deviation
0,283.356510,291.813104,47.945573,1.357856e+13,1.340867e+13,2.551524e+12,5.154785e+10,5.176856e+10,1.306473e+09,7.500794e+23,...,1.549630e+22,3687.642261,3700.402014,137.146720,7.527113e+13,7.492170e+13,3.171958e+12,5.919520e+22,5.924058e+22,2.473761e+21
1,66.619653,60.367148,27.764524,8.933433e+12,8.835084e+12,8.927245e+11,2.140709e+10,2.124389e+10,6.286775e+08,3.718566e+23,...,8.245439e+21,1888.013673,1884.090816,74.612870,3.993048e+13,4.043411e+13,1.913115e+12,2.053279e+22,2.071389e+22,6.880762e+20
2,127.237679,126.322234,13.837997,1.424802e+12,1.573723e+12,6.779822e+11,1.436737e+10,1.425573e+10,3.057145e+08,2.293910e+23,...,3.866595e+21,916.024647,920.941149,24.424215,1.858903e+13,1.862647e+13,6.995325e+11,1.560623e+22,1.536876e+22,7.693496e+20
3,746.403673,737.612053,54.688547,4.978973e+13,4.852730e+13,4.374883e+12,6.333512e+10,6.292214e+10,1.559654e+09,1.162628e+24,...,1.892603e+22,4193.908810,4180.870136,150.727003,7.960712e+13,8.000210e+13,2.077953e+12,5.395322e+22,5.343975e+22,2.196697e+21
4,23.520698,23.577261,14.918347,8.099298e+12,8.163373e+12,1.108614e+12,1.902392e+10,1.929112e+10,1.262244e+09,3.408467e+23,...,1.421864e+22,1749.004530,1747.972946,114.516238,3.763947e+13,3.740820e+13,2.997261e+12,1.853517e+22,1.859200e+22,1.673665e+21


In [103]:
# Adding target feature to data frame
extracted_features['Target'] = Y_target
extracted_features.head()
#extracted_featuresMean.shape

,ABSNJZH__median,ABSNJZH__mean,ABSNJZH__standard_deviation,SAVNCPP__median,SAVNCPP__mean,SAVNCPP__standard_deviation,TOTBSQ__median,TOTBSQ__mean,TOTBSQ__standard_deviation,TOTPOT__median,...,TOTUSJH__median,TOTUSJH__mean,TOTUSJH__standard_deviation,TOTUSJZ__median,TOTUSJZ__mean,TOTUSJZ__standard_deviation,USFLUX__median,USFLUX__mean,USFLUX__standard_deviation,Target
0,283.356510,291.813104,47.945573,1.357856e+13,1.340867e+13,2.551524e+12,5.154785e+10,5.176856e+10,1.306473e+09,7.500794e+23,...,3687.642261,3700.402014,137.146720,7.527113e+13,7.492170e+13,3.171958e+12,5.919520e+22,5.924058e+22,2.473761e+21,1
1,66.619653,60.367148,27.764524,8.933433e+12,8.835084e+12,8.927245e+11,2.140709e+10,2.124389e+10,6.286775e+08,3.718566e+23,...,1888.013673,1884.090816,74.612870,3.993048e+13,4.043411e+13,1.913115e+12,2.053279e+22,2.071389e+22,6.880762e+20,1
2,127.237679,126.322234,13.837997,1.424802e+12,1.573723e+12,6.779822e+11,1.436737e+10,1.425573e+10,3.057145e+08,2.293910e+23,...,916.024647,920.941149,24.424215,1.858903e+13,1.862647e+13,6.995325e+11,1.560623e+22,1.536876e+22,7.693496e+20,1
3,746.403673,737.612053,54.688547,4.978973e+13,4.852730e+13,4.374883e+12,6.333512e+10,6.292214e+10,1.559654e+09,1.162628e+24,...,4193.908810,4180.870136,150.727003,7.960712e+13,8.000210e+13,2.077953e+12,5.395322e+22,5.343975e+22,2.196697e+21,1
4,23.520698,23.577261,14.918347,8.099298e+12,8.163373e+12,1.108614e+12,1.902392e+10,1.929112e+10,1.262244e+09,3.408467e+23,...,1749.004530,1747.972946,114.516238,3.763947e+13,3.740820e+13,2.997261e+12,1.853517e+22,1.859200e+22,1.673665e+21,1


In [104]:

# splitting dataframe by row index
df_train = extracted_features.iloc[:9289,:]
df_test = extracted_features.iloc[9289:19667,:]
df_validate = extracted_features.iloc[19667:30216,:]


Y_train= df_train['Target']
X_train= df_train.drop('Target', axis=1)

y_test = df_test['Target']
X_test = df_test.drop('Target', axis=1)

y_valid = df_validate['Target']
X_valid = df_validate.drop('Target', axis=1)

In [105]:
print(X_train.shape, Y_train.shape, X_test.shape, y_test.shape, X_valid.shape, y_valid.shape)

(9289, 21) (9289,) (10378, 21) (10378,) (10549, 21) (10549,)


In [92]:

unique, counts = np.unique(y_valid, return_counts=True)

print(np.asarray((unique, counts)).T) 

[['0' 9125]
 ['1' 1424]]


In [114]:
# Random forest classifier
rfc = RandomForestClassifier(n_estimators=200, random_state= 7)
rfc_model=rfc.fit(X_train, Y_train)
pred_rfc = rfc_model.predict(X_test)


print(classification_report(y_test, pred_rfc))

cm_rfc = confusion_matrix(y_test, pred_rfc)

              precision    recall  f1-score   support

           0       0.94      0.97      0.96      8977
           1       0.78      0.59      0.67      1401

    accuracy                           0.92     10378
   macro avg       0.86      0.78      0.81     10378
weighted avg       0.92      0.92      0.92     10378



In [145]:
cm_rfc

array([[8746,  231],
       [ 576,  825]], dtype=int64)

In [111]:
def accuracy_measure(TN,FP,FN,TP):
    P = TP + FN
    N = FP + TN
    amdf = pd.DataFrame()
    amdf['TP'] = [TP]
    amdf['FP'] = FP
    amdf['TN'] = TN
    amdf['FN'] = FN
    amdf['P'] = P
    amdf['N'] = N
    return(amdf)

In [112]:
def evaluate(amdf):
    tss = (amdf.TP / (amdf.TP + amdf.FN)) - (amdf.FP / (amdf.FP + amdf.TN))
    hss = (2 * ((amdf.TP * amdf.TN) - (amdf.FN * amdf.FP)) / (amdf.P * (amdf.FN + amdf.TN)
                                                              + amdf.N * (amdf.TP + amdf.FP)))
    precXM = amdf.TP / (amdf.TP + amdf.FP)
    recallXM = amdf.TP / (amdf.TP + amdf.FN)
    precCBN = amdf.TN / (amdf.TN + amdf.FN)
    recallCBN = amdf.TN / (amdf.TN + amdf.FP)
    far = amdf.FP / (amdf.FP + amdf.TP)
    ch = (amdf.TP + amdf.FP) * (amdf.TP + amdf.FN) / (amdf.TP + amdf.FP + amdf.FN + amdf.TN)
    gss = (amdf.TP - ch) / (amdf.TP + amdf.FP + amdf.FN - ch)
    eval_metrics = pd.DataFrame()
    eval_metrics['TSS'] = [tss.iloc[0]]
    eval_metrics['HSS'] = hss.iloc[0]
    eval_metrics['GSS'] = gss.iloc[0]
    eval_metrics['Precision(Class 1)'] = precXM.iloc[0]
    eval_metrics['Recall(Class 1)'] = recallXM.iloc[0]
    eval_metrics['Precision(Class 0)'] = precCBN.iloc[0]
    eval_metrics['Recall( Class 0)'] = recallCBN.iloc[0]
    eval_metrics['FAR'] = far.iloc[0]
    return(eval_metrics)

In [115]:
df_cm = pd.DataFrame(cm_rfc)
df_cm

TN=df_cm.iloc[0,0]
FP=df_cm.iloc[0,1]
FN=df_cm.iloc[1,0]
TP=df_cm.iloc[1,1]



In [116]:
amdf=accuracy_measure(TN,FP,FN,TP)
Test_evaluation_report=evaluate(amdf)
Test_evaluation_report

,TSS,HSS,GSS,Precision(Class 1),Recall(Class 1),Precision(Class 0),Recall( Class 0),FAR
0,0.563133,0.628434,0.458187,0.78125,0.588865,0.938211,0.974268,0.21875


In [119]:
# Validating Random forest classifier
y_pred_validate = rfc_model.predict(X_valid)

cm_rfc_validate = confusion_matrix(y_pred_validate, y_valid)
print("\nclassification_report \n",classification_report(y_valid,y_pred_validate))


classification_report 
               precision    recall  f1-score   support

           0       0.93      0.96      0.94      9125
           1       0.67      0.54      0.60      1424

    accuracy                           0.90     10549
   macro avg       0.80      0.75      0.77     10549
weighted avg       0.89      0.90      0.90     10549



In [120]:
df_cm = pd.DataFrame(cm_rfc_validate)
df_cm

TN=df_cm.iloc[0,0]
FP=df_cm.iloc[0,1]
FN=df_cm.iloc[1,0]
TP=df_cm.iloc[1,1]

amdf=accuracy_measure(TN,FP,FN,TP)
Test_evaluation_report=evaluate(amdf)
Test_evaluation_report

,TSS,HSS,GSS,Precision(Class 1),Recall(Class 1),Precision(Class 0),Recall( Class 0),FAR
0,0.595687,0.540249,0.370097,0.539326,0.665511,0.957699,0.930176,0.460674


In [ ]:
#Now lets try to do some evaluation for random forest model using cross validation.
rfc_eval = cross_val_score(estimator = rfc_model, X = X_train, y = Y_train, cv = 10)
rfc_eval.mean()

0.9818078161538176

In [122]:
# SVC
svc = SVC()
svc_model=svc.fit(X_train, Y_train)
pred_svc = svc_model.predict(X_test)

print(classification_report(y_test, pred_svc))

cm_svc = confusion_matrix(y_test, pred_svc)


              precision    recall  f1-score   support

           0       0.91      0.98      0.94      8977
           1       0.76      0.36      0.49      1401

    accuracy                           0.90     10378
   macro avg       0.83      0.67      0.72     10378
weighted avg       0.89      0.90      0.88     10378



In [146]:
cm_svc

array([[8817,  160],
       [ 899,  502]], dtype=int64)

In [123]:
df_cm = pd.DataFrame(cm_svc)
df_cm

TN=df_cm.iloc[0,0]
FP=df_cm.iloc[0,1]
FN=df_cm.iloc[1,0]
TP=df_cm.iloc[1,1]

amdf=accuracy_measure(TN,FP,FN,TP)
Test_evaluation_report=evaluate(amdf)
Test_evaluation_report

,TSS,HSS,GSS,Precision(Class 1),Recall(Class 1),Precision(Class 0),Recall( Class 0),FAR
0,0.340492,0.437977,0.280391,0.758308,0.358315,0.907472,0.982177,0.241692


In [124]:
#SVC Validate
y_pred_validate = svc_model.predict(X_valid)

cm_svc_validate = confusion_matrix(y_pred_validate, y_valid)
print("\nclassification_report \n",classification_report(y_valid,y_pred_validate))


classification_report 
               precision    recall  f1-score   support

           0       0.90      0.96      0.93      9125
           1       0.60      0.35      0.44      1424

    accuracy                           0.88     10549
   macro avg       0.75      0.65      0.69     10549
weighted avg       0.86      0.88      0.87     10549



In [125]:
df_cm = pd.DataFrame(cm_svc_validate)
df_cm

TN=df_cm.iloc[0,0]
FP=df_cm.iloc[0,1]
FN=df_cm.iloc[1,0]
TP=df_cm.iloc[1,1]

amdf=accuracy_measure(TN,FP,FN,TP)
Test_evaluation_report=evaluate(amdf)
Test_evaluation_report

,TSS,HSS,GSS,Precision(Class 1),Recall(Class 1),Precision(Class 0),Recall( Class 0),FAR
0,0.503463,0.376775,0.232115,0.345506,0.599269,0.963945,0.904194,0.654494


In [ ]:
#Now lets try to do some evaluation for SVC  using cross validation.
svc_eval = cross_val_score(estimator = svc_model, X = X_train, y = Y_train, cv = 10)
svc_eval.mean()


0.9015013130544522

In [128]:
#initialize a tree based classification

dt_clf = DecisionTreeClassifier(max_depth = 2, criterion='entropy')
dt_Model = dt_clf.fit(X_train, Y_train)
dt_y_pred = dt_Model.predict(X_test)

print(classification_report(y_test, dt_y_pred))

cm_dt = confusion_matrix(y_test, dt_y_pred)

              precision    recall  f1-score   support

           0       0.94      0.97      0.95      8977
           1       0.75      0.58      0.65      1401

    accuracy                           0.92     10378
   macro avg       0.84      0.78      0.80     10378
weighted avg       0.91      0.92      0.91     10378



In [147]:
cm_dt

array([[8699,  278],
       [ 587,  814]], dtype=int64)

In [129]:
df_cm = pd.DataFrame(cm_dt)
df_cm

TN=df_cm.iloc[0,0]
FP=df_cm.iloc[0,1]
FN=df_cm.iloc[1,0]
TP=df_cm.iloc[1,1]

amdf=accuracy_measure(TN,FP,FN,TP)
Test_evaluation_report=evaluate(amdf)
Test_evaluation_report

,TSS,HSS,GSS,Precision(Class 1),Recall(Class 1),Precision(Class 0),Recall( Class 0),FAR
0,0.550046,0.60649,0.435225,0.745421,0.581014,0.936787,0.969032,0.254579


In [137]:
#Decision tree Validate
y_pred_validate = dt_Model.predict(X_valid)

cm_dt_validate = confusion_matrix(y_pred_validate, y_valid) 
print("\nclassification_report \n",classification_report(y_valid,y_pred_validate))


classification_report 
               precision    recall  f1-score   support

           0       0.95      0.94      0.94      9125
           1       0.62      0.65      0.64      1424

    accuracy                           0.90     10549
   macro avg       0.78      0.80      0.79     10549
weighted avg       0.90      0.90      0.90     10549



In [140]:
df_cm = pd.DataFrame(cm_dt_validate) 

df_cm

TN=df_cm.iloc[0,0]
FP=df_cm.iloc[0,1]
FN=df_cm.iloc[1,0]
TP=df_cm.iloc[1,1]

amdf=accuracy_measure(TN,FP,FN,TP)
Test_evaluation_report=evaluate(amdf)
Test_evaluation_report

,TSS,HSS,GSS,Precision(Class 1),Recall(Class 1),Precision(Class 0),Recall( Class 0),FAR
0,0.568067,0.578777,0.407238,0.651685,0.622819,0.938411,0.945248,0.348315


In [ ]:
#Now lets try to do some evaluation for SVC  using cross validation.
dt_eval = cross_val_score(estimator = dt_Model, X = X_train, y = Y_train, cv = 10)
dt_eval.mean()

0.8650016471177759

In [133]:
# K nearest neighbor

knClassifier = KNeighborsClassifier()
knModel = knClassifier.fit(X_train, Y_train)
kn_Y_Pred = knModel.predict(X_test)

print(classification_report(y_test, kn_Y_Pred))

cm_knearest = confusion_matrix(y_test, kn_Y_Pred)

              precision    recall  f1-score   support

           0       0.92      0.95      0.93      8977
           1       0.59      0.46      0.52      1401

    accuracy                           0.88     10378
   macro avg       0.76      0.70      0.73     10378
weighted avg       0.87      0.88      0.88     10378



In [148]:
cm_knearest

array([[8539,  438],
       [ 761,  640]], dtype=int64)

In [134]:
df_cm = pd.DataFrame(cm_knearest)
df_cm

TN=df_cm.iloc[0,0]
FP=df_cm.iloc[0,1]
FN=df_cm.iloc[1,0]
TP=df_cm.iloc[1,1]

amdf=accuracy_measure(TN,FP,FN,TP)
Test_evaluation_report=evaluate(amdf)
Test_evaluation_report

,TSS,HSS,GSS,Precision(Class 1),Recall(Class 1),Precision(Class 0),Recall( Class 0),FAR
0,0.408025,0.451998,0.291988,0.593692,0.456817,0.918172,0.951209,0.406308


In [142]:
#K nearest Validate
y_pred_validate = knModel.predict(X_valid)

cm_knearest_Validate = confusion_matrix(y_pred_validate, y_valid)
print("\nclassification_report \n",classification_report(y_valid,y_pred_validate))


classification_report 
               precision    recall  f1-score   support

           0       0.94      0.93      0.93      9125
           1       0.57      0.59      0.58      1424

    accuracy                           0.89     10549
   macro avg       0.75      0.76      0.76     10549
weighted avg       0.89      0.89      0.89     10549



In [144]:
df_cm = pd.DataFrame(cm_knearest_Validate)
df_cm

TN=df_cm.iloc[0,0]
FP=df_cm.iloc[0,1]
FN=df_cm.iloc[1,0]
TP=df_cm.iloc[1,1]

amdf=accuracy_measure(TN,FP,FN,TP)
Test_evaluation_report=evaluate(amdf)
Test_evaluation_report

,TSS,HSS,GSS,Precision(Class 1),Recall(Class 1),Precision(Class 0),Recall( Class 0),FAR
0,0.509906,0.514802,0.346621,0.587781,0.574468,0.932055,0.935438,0.412219
